In [ ]:
# Step 1: Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Step 2: Load and Preprocess Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Show a 5x5 grid of MNIST digit samples
plt.figure(figsize=(10, 5))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f"Label: {y_train[i]}")
    plt.axis('off')
plt.suptitle("Sample MNIST Digits", y=1.02)
plt.tight_layout()
plt.show()


# Bar chart showing how many samples per digit
import collections

mnist_counts = collections.Counter(y_train)
plt.bar(mnist_counts.keys(), mnist_counts.values(), color="orange")
plt.title("MNIST Label Distribution")
plt.xlabel("Digit")
plt.ylabel("Count")
plt.xticks(range(10))
plt.show()


In [ ]:
# Step 3: Logistic Regression Baseline (Flattened input)
x_train_flat = x_train.reshape(len(x_train), -1)
x_test_flat = x_test.reshape(len(x_test), -1)
logreg = LogisticRegression(max_iter=1000)
logreg.fit(x_train_flat, y_train)
y_pred_log = logreg.predict(x_test_flat)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log))

In [ ]:
from sklearn.metrics import classification_report

print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_log, digits=4))


In [ ]:
# Step 4: Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(x_train)

In [ ]:
# Step 5: Build CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [ ]:
# Step 6: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
early_stop = EarlyStopping(patience=3, restore_best_weights=True)
model_ckpt = ModelCheckpoint('best_emnist_model.keras', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
callbacks = [early_stop, reduce_lr, model_ckpt]

In [ ]:
# Step 7: Train the Model with Augmentation
history = model.fit(datagen.flow(x_train, y_train_cat, batch_size=64),
                    validation_data=(x_test, y_test_cat),
                    epochs=25,
                    callbacks=callbacks)

In [ ]:
# Step 8: Evaluate the Model
test_loss, test_acc = model.evaluate(x_test, y_test_cat)
print(f"CNN Test Accuracy: {test_acc:.4f}")

In [ ]:
# Step 9: Accuracy/Loss Plots
plt.figure()
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Accuracy vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure()
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Loss vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Step 10: Confusion Matrix and Classification Report
y_pred_cnn = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, y_pred_cnn)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cbar=False)
plt.title('CNN Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

print("CNN Classification Report:")
print(classification_report(y_test, y_pred_cnn, digits=4))

In [ ]:
# Step 11: Visualize First Conv2D Filters
filters, _ = model.layers[0].get_weights()
filters = (filters - filters.min()) / (filters.max() - filters.min())
plt.figure(figsize=(8,4))
for i in range(6):
    plt.subplot(1,6,i+1)
    plt.imshow(filters[:, :, 0, i], cmap='gray')
    plt.axis('off')
plt.suptitle('First-layer convolutional filters')
plt.show()

In [ ]:


# Step 12: Misclassified Examples
mis = np.where(y_pred_cnn != y_test)[0][:25]
plt.figure(figsize=(10,10))
for idx, wrong in enumerate(mis):
    ax = plt.subplot(5,5,idx+1)
    plt.imshow(x_test[wrong].reshape(28,28), cmap='gray')
    plt.title(f'True:{y_test[wrong]} Pred:{y_pred_cnn[wrong]}')
    plt.axis('off')
plt.tight_layout()
plt.show()


## 🎯 F1 Score Evaluation

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")